In [1]:
import sys
sys.path.append("/workspace/QuantLab")

import pandas as pd
from RQCharts.main import RQCharts
#from talib import abstract
from RQAlpha import RTools
import matplotlib.pyplot as plt
import numpy as np

df1=RQCharts()
df1minute = df1.add_crypto("bitcoin", resolution="5m")
df1minute["time"]=pd.to_datetime(df1minute["Time"],format='%d.%m.%Y %H:%M:%S')
df1minute.set_index("time", inplace=True)
df = df

"""
df = df1minute.resample('5min').agg({
    'Open': 'first',       # First value in 5-minute window
    'High': 'max',         # Max value in 5-minute window
    'Low': 'min',          # Min value in 5-minute window
    'Close': 'last',       # Last value in 5-minute window
    'Volume': 'sum',       # Sum of volume in 5-minute window
})
"""


df.dropna(subset=['Close'], inplace=True)  # Removes rows where 'close' is NaN
df=df[df['High']!=df['Low']]

df["Close"] = df["Close"].astype(float)
df["Open"] = df["Open"].astype(float)
df["High"] = df["High"].astype(float)
df["Low"] = df["Low"].astype(float)
df["Volume"] = df["Volume"].astype(float)

tools = RTools()

df.index = pd.to_datetime(df.index, errors="coerce")
df['ATR'] = tools.ATR(df, window = 20)
df['RSI']=tools.RSI(df["Close"], period =15)
df["STD"] = tools.STD(df['Close'], 20)
df["upperband"], df["middleband"], df["lowerband"], df['pband'], df['wband'] = tools.BBANDS(df= df, window=20, std = 2.0)
df['ATR'] = tools.ATR(df, window = 20)

window_list = [5, 8, 14, 20, 30, 50, 100, 150, 200]


for w in window_list:
    df[f'EMA_{w}'] = tools.EMA(df, window=w)
    df[f'VWAP_{w}'] = tools.VWAP(df, window = w)
    
df.dropna()
df


OperationalError: could not translate host name "db1" to address: Temporary failure in name resolution


In [9]:
#STRATEGY 1.0 = unchanged version

df = df.tail(100000)
VWAPsignal = [0]*len(df)
backcandles = 3

for row in range(backcandles, len(df)):
        if max(df.Open[row], df.Close[row])>=df.VWAP_30[row]:
            VWAPsignal[row]=2
        if min(df.Open[row], df.Close[row])<=df.VWAP_30[row]:
            VWAPsignal[row] = 1
df['VWAPSignal'] = VWAPsignal

def TotalSignal(l):
    if (df.VWAPSignal[l]==2
        and df.Low[l]<=df.lowerband[l]
        and df.RSI[l]<45):
            return 1
    if (df.VWAPSignal[l]==1
        and df.High[l]>=df.upperband[l]
        and df.RSI[l]>55):
            return -1
    return 0
        
TotSignal = [0]*len(df)
for row in range(backcandles, len(df)): #careful backcandles used previous cell
    TotSignal[row] = TotalSignal(row)
df['TotalSignal'] = TotSignal
print(df[df.TotalSignal!=0].count())


NameError: name 'df' is not defined

In [9]:
#STRATEGY 2.0 = A RSI Regression strategy with EMA trend confirmation and bollinger/VWAP extreme price signals for higher odss. 


ema_rsi_signal = [0]*len(df)


fastEMA = 8
slowEMA = 30

ema_rsi_signal = np.where(
    (df[f"EMA.{fastEMA}"] >= df[f"EMA.{slowEMA}"]) & (df["RSI"] < 31),1,  
    np.where(
        (df[f"EMA.{fastEMA}"] < df[f"EMA.{slowEMA}"]) & (df["RSI"] > 69),-1, 0)
)

   
df["ema_rsi_signal"] = ema_rsi_signal

print(df[df["ema_rsi_signal"]!=0].count())

Open              1
High              1
Low               1
Close             1
Volume            1
ATR               1
RSI               1
STD               1
upperband         1
middleband        1
lowerband         1
pband             1
wband             1
EMA.5             1
VWAP.5            1
EMA.8             1
VWAP.8            1
EMA.14            1
VWAP.14           1
EMA.20            1
VWAP.20           1
EMA.30            1
VWAP.30           1
EMA.50            1
VWAP.50           1
EMA.100           1
VWAP.100          1
EMA.150           1
VWAP.150          1
EMA.200           1
VWAP.200          1
ema_rsi_signal    1
dtype: int64


In [10]:
#SIGNAL PLOTTING
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime

import numpy as np
def pointposbreak(x):
    if x['TotalSignal']==1:
        return x['High']+1e-4
    elif x['TotalSignal']==2:
        return x['Low']-1e-4
    else:
        return np.nan

df['pointposbreak'] = df.apply(lambda row: pointposbreak(row), axis=1)

st=16500
dfpl = df[st:st+5200]
dfpl.reset_index(inplace=True)
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close']),
                go.Scatter(x=dfpl.index, y=dfpl.VWAP, 
                           line=dict(color='blue', width=1), 
                           name="VWAP"), 
                go.Scatter(x=dfpl.index, y=dfpl['lowerband'], 
                           line=dict(color='green', width=1), 
                           name="BBL"),
                go.Scatter(x=dfpl.index, y=dfpl['upperband'], 
                           line=dict(color='green', width=1), 
                           name="BBU")])

fig.add_scatter(x=dfpl.index, y=dfpl['pointposbreak'], mode="markers",
                marker=dict(size=10, color="MediumPurple"),
                name="Signal")
fig.show()

/tmp/ipykernel_665173/2342590926.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['pointposbreak'] = df.apply(lambda row: pointposbreak(row), axis=1)


AttributeError: 'DataFrame' object has no attribute 'VWAP'

In [11]:
#BACKTESTING
from backtesting import Strategy
from backtesting import Backtest

df = df[30:]

def SIGNAL():
    return df.TotalSignal

class MyStrat(Strategy):
    TPSLRatio = 4
    initsize = 0.5 

    def init(self):
        super().init()
        self.mysize = self.initsize  
        self.signal1 = self.I(SIGNAL)

    def next(self):
        super().next()
        

        if len(self.data.ATR) < 2 or np.isnan(self.data.ATR[-1]):
            return

        slatr = 1.5 * self.data.ATR[-1]

      
        if len(self.trades) > 0:
            if self.trades[-1].is_long and self.data.RSI[-1] >= 80:
                self.trades[-1].close()
            elif self.trades[-1].is_short and self.data.RSI[-1] <= 20:
                self.trades[-1].close()

       
        signal = self.signal1[-1] if len(self.signal1) > 1 else None

       
        if signal == 1 and len(self.trades) < 3:
            sl1 = self.data.Close[-1] - slatr
            tp1 = self.data.Close[-1] + slatr * self.TPSLRatio
            tp2 = self.data.RSI[-1] >= 70
            self.buy(sl=sl1, tp=tp1, size=self.mysize)

       
        if signal == -1 and len(self.trades) < 3 :
            sl1 = self.data.Close[-1] + slatr
            tp1 = self.data.Close[-1] - slatr * self.TPSLRatio
            tp2 = self.data.RSI[-1] <= 33
            self.sell(sl=sl1, tp=tp1, size=self.mysize)



bt = Backtest(df, MyStrat, cash=10000, margin = 1/10, commission=0.00)
stat = bt.run()
bt.plot(
results=None,
filename="strat1.0_SPY_run2",
plot_width=None,
plot_equity=True,
plot_return=False,
plot_pl=True,
plot_volume=True,
plot_drawdown=False,
plot_trades=True,
smooth_equity=False,
relative_equity=True,
superimpose=False,
resample=True,
reverse_indicators=False,
show_legend=True,
open_browser=True)
stat



/workspace/.venv/lib/python3.12/site-packages/backtesting/_plotting.py:53: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

/tmp/ipykernel_665173/1336193968.py:54: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(df, MyStrat, cash=10000, margin = 1/10, commission=0.00)
/workspace/.venv/lib/python3.12/site-packages/backtesting/_plotting.py:125: UserWarning: Data contains too many candlesticks to plot; downsampling to '2h'. See `Backtest.plot(resample=...)`
  warnings.warn(f"Data contains too many candlesticks to plot; downsampling to {freq!r}. "


Start                     2023-10-18 12:50:00
End                       2025-01-26 19:00:00
Duration                    466 days 06:10:00
Exposure Time [%]                     0.64119
Equity Final [$]                  11149.19426
Equity Peak [$]                    12868.5938
Return [%]                           11.49194
Buy & Hold Return [%]                262.1507
Return (Ann.) [%]                    10.40841
Volatility (Ann.) [%]                16.21538
CAGR [%]                               8.8889
Sharpe Ratio                          0.64189
Sortino Ratio                         1.43752
Calmar Ratio                            0.779
Max. Drawdown [%]                   -13.36121
Avg. Drawdown [%]                    -1.63241
Max. Drawdown Duration      332 days 05:15:00
Avg. Drawdown Duration       14 days 11:23:00
# Trades                                   51
Win Rate [%]                          25.4902
Best Trade [%]                        1.89552
Worst Trade [%]                   

In [9]:
# GPT BACKTEST

import numpy as np
import datetime
from backtesting import Strategy, Backtest

def SIGNAL():
    return df.ema_rsi_signal  # Example signal function

class MyStrat2(Strategy):
    # Let backtesting.py treat x and rr as tunable parameters
    # (so you can specify them when you run bt.run(x=..., rr=...))
    parameters = dict(
        x=1.5,
        rr=1.9,
        initsize=0.33,
    )

    def init(self):
        self.signal1 = self.I(SIGNAL)
        self.mysize = self.initsize
        
    def next(self):
        super().next()

        # Current bar index: backtesting.py sets this as len(self.data) - 1
        current_bar = len(self.data) - 1
        # We'll assume each bar is 5 minutes; adjust if your data is otherwise
        bar_length_minutes = 5

 
        day_bar_count = 78  # for a 9:30-16:00 session, 5-min bars
        bar_in_day = current_bar % day_bar_count  # not always perfect, but example

        # If bar_in_day < 2 => first 10 min, bar_in_day > 75 => last 10 min
        skip_new_entries = (bar_in_day < 2 or bar_in_day > 75)

        # Make sure we have enough ATR data
        if len(self.data.ATR) < 2 or np.isnan(self.data.ATR[-1]):
            return

        # Pull RSI, ATR, etc.
        slatr = self.x * self.data.ATR[-1]
        two_atr = (self.x + self.rr) * self.data.ATR[-1]
        rsi_val = self.data.RSI[-1]
        current_price = self.data.Close[-1]

    
        for trade in self.trades:
            # `exit_bar` is None => trade is still active
            if trade.exit_bar is None:
                bars_in_trade = current_bar - trade.entry_bar
                minutes_in_trade = bars_in_trade * bar_length_minutes

                # trade.is_short => True if size < 0
                if minutes_in_trade < 20:
                    # If < 20 minutes in tradeF
                    if trade.is_long:
                        # If current price >= (entry_price + 2×ATR), exit
                        if current_price >= (trade.entry_price + two_atr):
                            trade.close()
                    else:  # trade.is_short
                        # If current price <= (entry_price - 2×ATR), exit
                        if current_price <= (trade.entry_price - two_atr):
                            trade.close()
                else:
                    # If ≥ 20 minutes in trade, exit based on RSI alone
                    if trade.is_long and rsi_val > 69:
                        trade.close()
                    elif trade.is_short and rsi_val < 30:
                        trade.close()

        if len(self.trades) > 0:
            last_trade = self.trades[-1]
            if last_trade.exit_bar is None:  # trade is still open
                if last_trade.is_long and rsi_val >= 70:
                    last_trade.close()
                elif last_trade.is_short and rsi_val <= 33:
                    last_trade.close()


        # We'll only open up to 3 trades total at once
        if len(self.trades) < 3 and not skip_new_entries:
            signal = self.signal1[-1] if len(self.signal1) > 0 else None

            if signal == 1:
                self.buy(size=self.mysize)

            elif signal == -1:
                self.sell(size=self.mysize)



bt = Backtest(df, MyStrat2, cash=10000, margin = 1/10, commission=0.00)
stats = bt.run()
print(stats)
bt.plot()


AttributeError: 'MyStrat2' object has no attribute 'initsize'

In [20]:
#OPTIMIZING
import numpy as np

bt = Backtest(df, MyStrat, cash=10000, margin=1/10, commission=0.00)

stat = bt.optimize(
    TPSLRatio=np.arange(1.5, 4.1, 0.10).tolist(),
    initsize=np.arange(0.3, 1.00, 0.05).tolist(),
    maximize='Sharpe Ratio',
    method='grid',
    max_tries=None,
    constraint=None,
    return_heatmap=True,
    return_optimization=False,
    random_state=None,
    )
print(stat)



/workspace/.venv/lib/python3.12/site-packages/backtesting/backtesting.py:1592: UserWarning: Searching for best of 364 configurations.
  output = _optimize_grid()


(Start                     2023-12-26 14:30:00
End                       2025-02-07 15:55:00
Duration                    409 days 01:25:00
Exposure Time [%]                    51.95356
Equity Final [$]                  14272.04199
Equity Peak [$]                   15175.57717
Return [%]                           42.72042
Buy & Hold Return [%]                26.30992
Return (Ann.) [%]                    37.73284
Volatility (Ann.) [%]                67.33063
CAGR [%]                             24.50043
Sharpe Ratio                          0.56041
Sortino Ratio                         1.12248
Calmar Ratio                          1.20962
Max. Drawdown [%]                   -31.19394
Avg. Drawdown [%]                    -1.71723
Max. Drawdown Duration      135 days 02:30:00
Avg. Drawdown Duration        3 days 12:28:00
# Trades                                 1640
Win Rate [%]                         29.02439
Best Trade [%]                        2.46957
Worst Trade [%]                  